# LeNet++ implementation

the previous tutorial is using softmax regression to recognize MNIST digits.

The tutorial for this notebook is here:

[Building a Multilayer Convolutional Network](https://www.tensorflow.org/versions/r0.10/tutorials/mnist/pros/index.html#deep-mnist-for-experts#content)

In [1]:
import tensorflow as tf

In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

In [3]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [4]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [5]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

In [6]:
# parametric rectified linear unit
def prelu(x, alphas):
    return tf.nn.relu(x) + tf.mul(alphas, tf.sub(x, tf.abs(x)))

In [7]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [8]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [9]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
alphas_conv1 = bias_variable([32])
layer_conv_1 = prelu(conv2d(x_image, W_conv1) + b_conv1, alphas_conv1)

In [10]:
# W_conv2 = weight_variable([5, 5, 32, 1])
# b_conv2 = bias_variable([32])
# alphas_conv2 = bias_variable([32])
# layer_conv_2 = prelu(conv2d(layer_conv_1, W_conv2) + b_conv2, alphas_conv2)
stage_1_pool = max_pool_2x2(layer_conv_1)

In [11]:
W_conv3 = weight_variable([5, 5, 32, 64])
b_conv3 = bias_variable([64])
alphas_conv3 = bias_variable([64])
layer_conv_3 = prelu(conv2d(layer_conv_1, W_conv3) + b_conv3, alphas_conv3)

# W_conv4 = weight_variable([5, 5, 64, 1])
# b_conv4 = bias_variable([64])
# alphas_conv4 = bias_variable([64])
# layer_conv_4 = prelu(conv2d(layer_conv_3, W_conv4) + b_conv4, alphas_conv4)
stage_2_pool = max_pool_2x2(layer_conv_3)

## Stage 3:

In [12]:
W_conv5 = weight_variable([5, 5, 64, 128])
b_conv5 = bias_variable([128])
alphas_conv5 = bias_variable([128])
layer_conv_5 = prelu(conv2d(layer_conv_3, W_conv5) + b_conv5, alphas_conv5)

# W_conv6 = weight_variable([5, 5, 128, 1])
# b_conv6 = bias_variable([128])
# alphas_conv6 = bias_variable([128])
# layer_conv_6 = prelu(conv2d(layer_conv_5, W_conv6) + b_conv6, alphas_conv6)

stage_3_pool = max_pool_2x2(layer_conv_5)

In [32]:
layer_conv_5.get_shape()

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(128)])

In [23]:
W_fc1 = weight_variable([14 * 14 * 128, 200])
b_fc1 = bias_variable([200])
alphas_fc1 = bias_variable([200])

stage_3_pool_flat = tf.reshape(stage_3_pool, [-1, 14 * 14 * 128])
h_fc1 = prelu(tf.matmul(stage_3_pool_flat, W_fc1) + b_fc1, alphas_fc1)

In [24]:
# keep_prob = tf.placeholder(tf.float32)
# h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
# W_fc2 = weight_variable([2, 10])
# b_fc2 = bias_variable([10])

# y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [25]:
W_fc2 = weight_variable([200, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)

In [26]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [27]:
sess = tf.InteractiveSession()

In [30]:
%%time
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(200000):
  batch = mnist.train.next_batch(10)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1]}) # , keep_prob: 1.0
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})#, keep_prob: 0.5

ResourceExhaustedError: OOM when allocating tensor with shape[25088,200]
	 [[Node: truncated_normal_5/TruncatedNormal = TruncatedNormal[T=DT_INT32, dtype=DT_FLOAT, seed=0, seed2=0, _device="/job:localhost/replica:0/task:0/gpu:0"](truncated_normal_5/shape)]]
Caused by op u'truncated_normal_5/TruncatedNormal', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 589, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 405, in start
    ioloop.IOLoop.instance().start()
  File "/usr/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 160, in start
    super(ZMQIOL

In [ ]:
%%time
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images[:250], y_: mnist.test.labels[:250], keep_prob: 1.0}))

In [ ]:
\
\


























